## IBM Capstone - WK3 submission (PART III)

This assignment creates a github account and pushes a iPython notebook to your github account

In [1]:
import os, pandas as pd, geocoder, numpy as np

In [2]:
#set working directory:
os.chdir("/Users/christinaorndorff/Documents/Fred's Stuff/Coursa/IBM_Capstone")

# PART I:
### For details on this code - please see the Part 1 Jupyter notebook

In [3]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)
canada_df = dfs[0]
canada_df = canada_df[canada_df.Borough != 'Not assigned']
canada_df = canada_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).reset_index()
canada_df['Neighbourhood'].replace('Not assigned',canada_df['Borough'],inplace = True)
canada_df.columns = ['PostalCode', 'Borough', 'Neighborhood']
canada_df.shape

(103, 3)

# PART II:
### For details on this code - please see the Part 2 Jupyter notebook

In [4]:
canada_df_coord = pd.read_csv('Geospatial_Coordinates.csv')
canada_df_coord.columns = ['PostalCode', 'Latitude', 'Longitude']
can_df = pd.merge(canada_df, canada_df_coord, on = 'PostalCode')
can_df.shape

(103, 5)

# PART III:
### I followed the second lab for week 3 -- the code is pulled directly from the lab.

In [5]:
import requests, json, folium, geopy
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

### Retriving the coordinates for the Toronto base map

In [6]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent = "canada_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.653963, -79.387207.


In [7]:
neighborhoods = can_df
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Adding the boroughs and neighborhoods to the map

In [8]:
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], 
                                           neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

In [9]:
map_toronto

### Let's focus on Scarborough
First we will subset the data; then complete the map as we did above for Toronto.

In [10]:
#subset the data
scar_data = neighborhoods[neighborhoods['Borough'] == 'Scarborough'].reset_index(drop=True)
scar_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [11]:
address = 'Scarborough, Canada'

geolocator = Nominatim(user_agent = "canada_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [12]:
# create map of Scarborough, Canada using latitude and longitude values
map_scar = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scar_data['Latitude'], scar_data['Longitude'], 
                           scar_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scar) 
    
map_scar

### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [13]:
CLIENT_ID = 'GBTVHG14LE2WKWDFJYDTF4UTKWLDY5SAULYKHNHS5DVORM5A' # your Foursquare ID
CLIENT_SECRET = 'KWLNACCQGFTFG01N1530FPPZCGGTES4VK4AW3ZH4QDLIQU44' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [14]:
scar_data.loc[0, 'Neighborhood']

'Rouge,Malvern'

In [15]:
neighborhood_latitude = scar_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scar_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scar_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.806686299999996, -79.19435340000001.


### This is the top 100 venues within 500 miles of Scarbrough

In [16]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

results = requests.get(url).json()

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


In [19]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


### This code gets all venues for all the neighborhoods in Scarbrough

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
scar_venues = getNearbyVenues(names = scar_data['Neighborhood'],
                                   latitudes = scar_data['Latitude'],
                                   longitudes = scar_data['Longitude']
                                  )


Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge


In [22]:
print(scar_venues.shape)
scar_venues.head()

(92, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge,Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [23]:
scar_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,3,3,3,3,3,3
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",3,3,3,3,3,3
"Birch Cliff,Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clairlea,Golden Mile,Oakridge",10,10,10,10,10,10
"Clarks Corners,Sullivan,Tam O'Shanter",14,14,14,14,14,14
"Cliffcrest,Cliffside,Scarborough Village West",2,2,2,2,2,2
"Dorset Park,Scarborough Town Centre,Wexford Heights",6,6,6,6,6,6
"East Birchmount Park,Ionview,Kennedy Park",5,5,5,5,5,5


In [24]:
print('There are {} uniques categories.'.format(len(scar_venues['Venue Category'].unique())))

There are 52 uniques categories.


In [25]:
# one hot encoding
scar_onehot = pd.get_dummies(scar_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scar_onehot['Neighborhood'] = scar_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scar_onehot.columns[-1]] + list(scar_onehot.columns[:-1])
scar_onehot = scar_onehot[fixed_columns]

scar_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,...,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
scar_grouped = scar_onehot.groupby('Neighborhood').mean().reset_index()
scar_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,...,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.333333,0.0,0.0,...,0.0,0.000,0.000000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.000000
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.000,0.000000,0.333333,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000,0.000000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.000000
2,"Birch Cliff,Cliffside West",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000,0.000000,0.000000,0.25,0.0,0.000,0.000000,0.000000,0.000000
3,Cedarbrae,0.0,0.125,0.000000,0.125000,0.125000,0.0,0.000000,0.0,0.0,...,0.0,0.000,0.000000,0.000000,0.00,0.0,0.000,0.000000,0.125000,0.000000
4,"Clairlea,Golden Mile,Oakridge",0.0,0.000,0.000000,0.200000,0.000000,0.0,0.000000,0.2,0.1,...,0.0,0.000,0.000000,0.000000,0.00,0.1,0.000,0.000000,0.000000,0.000000
5,"Clarks Corners,Sullivan,Tam O'Shanter",0.0,0.000,0.000000,0.000000,0.071429,0.0,0.000000,0.0,0.0,...,0.0,0.000,0.000000,0.071429,0.00,0.0,0.000,0.000000,0.071429,0.000000
6,"Cliffcrest,Cliffside,Scarborough Village West",0.5,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000,0.000000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.000000
7,"Dorset Park,Scarborough Town Centre,Wexford He...",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000,0.000000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.166667
8,"East Birchmount Park,Ionview,Kennedy Park",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000,0.000000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.000000
9,"Guildwood,Morningside,West Hill",0.0,0.000,0.000000,0.000000,0.000000,0.0,0.125000,0.0,0.0,...,0.0,0.125,0.000000,0.000000,0.00,0.0,0.125,0.000000,0.000000,0.000000


In [27]:
num_top_venues = 5

for hood in scar_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scar_grouped[scar_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.33
1                     Lounge  0.33
2             Breakfast Spot  0.33
3                   Pharmacy  0.00
4         Light Rail Station  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                 venue  freq
0               Bakery  0.33
1                 Park  0.33
2           Playground  0.33
3  American Restaurant  0.00
4             Pharmacy  0.00


----Birch Cliff,Cliffside West----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25
3                   Café  0.25
4               Pharmacy  0.00


----Cedarbrae----
                venue  freq
0     Thai Restaurant  0.12
1              Bakery  0.12
2                Bank  0.12
3  Athletics & Sports  0.12
4    Hakka Restaurant  0.12


----Clairlea,Golden Mile,Oakridge----
           venue  freq
0         Bakery   0.2
1       Bus Line   0.2
2   Intersecti

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scar_grouped['Neighborhood']

for ind in np.arange(scar_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scar_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Breakfast Spot,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Bakery,Playground,Park,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,"Birch Cliff,Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant
3,Cedarbrae,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Vietnamese Restaurant,Department Store
4,"Clairlea,Golden Mile,Oakridge",Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant,Bus Station,Metro Station,Park,College Stadium,General Entertainment


## Clustering
### I used K-means clustering to group neighborhoods within Scarbrough

In [30]:
# set number of clusters
kclusters = 5

scar_grouped_clustering = scar_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scar_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 2, 1, 1, 1], dtype=int32)

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scar_merged = scar_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scar_merged = scar_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), 
                               on='Neighborhood')

scar_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,3.0,Bar,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,1.0,Intersection,Spa,Electronics Store,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Coffee Shop,Gas Station
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4.0,Coffee Shop,Indian Restaurant,Korean Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Vietnamese Restaurant,Department Store


### It looks like row 16 has a null value -- I will drop that row

In [32]:
null_columns = scar_merged.columns[scar_merged.isnull().any()]
print(scar_merged[scar_merged.isnull().any(axis=1)][null_columns].head())

    Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
16             NaN                   NaN                   NaN   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
16                   NaN                   NaN                   NaN   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
16                   NaN                   NaN                   NaN   

   9th Most Common Venue 10th Most Common Venue  
16                   NaN                    NaN  


In [33]:
scar_merged = scar_merged.dropna()

### Showing the clustering on a map

In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scar_merged['Latitude'], 
                                  scar_merged['Longitude'], 
                                  scar_merged['Neighborhood'], 
                                  scar_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Let's see how the k-means grouped the neighberhoods

In [35]:
scar_merged.loc[scar_merged['Cluster Labels'] == 0, 
                scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0.0,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store


In [36]:
scar_merged.loc[scar_merged['Cluster Labels'] == 1, 
                scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,1.0,Intersection,Spa,Electronics Store,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Coffee Shop,Gas Station
4,Scarborough,1.0,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Vietnamese Restaurant,Department Store
5,Scarborough,1.0,Playground,Convenience Store,Vietnamese Restaurant,Coffee Shop,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
6,Scarborough,1.0,Discount Store,Coffee Shop,Department Store,Convenience Store,Thai Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
7,Scarborough,1.0,Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant,Bus Station,Metro Station,Park,College Stadium,General Entertainment
9,Scarborough,1.0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant
10,Scarborough,1.0,Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Light Rail Station,Pet Store,Bank,Bar,Grocery Store,General Entertainment,Gas Station
11,Scarborough,1.0,Middle Eastern Restaurant,Auto Garage,Bakery,Shopping Mall,Sandwich Place,Breakfast Spot,Vietnamese Restaurant,College Stadium,General Entertainment,Gas Station
12,Scarborough,1.0,Latin American Restaurant,Lounge,Breakfast Spot,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
13,Scarborough,1.0,Pizza Place,Intersection,Fried Chicken Joint,Noodle House,Convenience Store,Pharmacy,Fast Food Restaurant,Shopping Mall,Bank,Italian Restaurant


In [37]:
scar_merged.loc[scar_merged['Cluster Labels'] == 2, 
                scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,2.0,American Restaurant,Motel,Indian Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


In [38]:
scar_merged.loc[scar_merged['Cluster Labels'] == 3, 
                scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3.0,Bar,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [39]:
scar_merged.loc[scar_merged['Cluster Labels'] == 4, 
                scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,4.0,Coffee Shop,Indian Restaurant,Korean Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
